In [1]:
import pandas as pd

In [2]:
train_data = pd.read_pickle('/data/sukhanna/cse258/processed_100/train_data.pkl')
val_data = pd.read_pickle('/data/sukhanna/cse258/processed_100/val_data.pkl')
test_data = pd.read_pickle('/data/sukhanna/cse258/processed_100/test_data.pkl')

id2item = pd.read_pickle('/data/sukhanna/cse258/processed_100/id2item.pkl')
item2id = pd.read_pickle('/data/sukhanna/cse258/processed_100/item2id.pkl')

id2user = pd.read_pickle('/data/sukhanna/cse258/processed_100/id2user.pkl')
user2id = pd.read_pickle('/data/sukhanna/cse258/processed_100/user2id.pkl')

In [3]:
import torch
from torch.utils.data import Dataset
import random

class FPMCDataset(Dataset):
    def __init__(self, train_data, val_data, test_data, num_items, n_neg=1):
        self.num_items = num_items
        self.n_neg = n_neg
        
        # 1. Prepare Storage for Transitions
        # Format: (user_idx, last_item_idx, current_item_idx)
        self.samples = []
        
        # 2. Prepare Exclusion Sets (Same as BPR)
        self.exclusion_rules = {}

        for u_idx in range(len(train_data)):
            # Extract sequence
            seq = [x['item_id'] for x in train_data[u_idx]['sequence']]
            val_target = val_data[u_idx]['target']['item_id']
            test_target = test_data[u_idx]['target']['item_id']
            
            # Build Exclusion Set (Train + Val + Test)
            full_history = set(seq)
            full_history.add(val_target)
            full_history.add(test_target)
            self.exclusion_rules[u_idx] = full_history
            
            # 3. Generate Sequential Transitions
            # We need at least 2 items to form a transition (A -> B)
            if len(seq) > 1:
                # Iterate from 0 to Length-2
                for i in range(len(seq) - 1):
                    last_item = seq[i]
                    curr_item = seq[i+1]
                    
                    self.samples.append((u_idx, last_item, curr_item))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        u_id, last_item_raw, pos_item_raw = self.samples[idx]
        
        # Negative Sampling
        # We sample 'n_neg' items that are NOT in the user's full history
        neg_samples = []
        for _ in range(self.n_neg):
            while True:
                neg_id_raw = random.randint(1, self.num_items)
                if neg_id_raw not in self.exclusion_rules[u_id]:
                    neg_samples.append(neg_id_raw - 1)
                    break
        
        # Return 0-indexed tensors
        # Note: We subtract 1 because input IDs are 1-based
        return (
            torch.tensor(u_id, dtype=torch.long),
            torch.tensor(last_item_raw - 1, dtype=torch.long),
            torch.tensor(pos_item_raw - 1, dtype=torch.long),
            torch.tensor(neg_samples, dtype=torch.long) # Shape: [n_neg]
        )

In [4]:
import torch
import torch.nn as nn

class FPMC(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64):
        """
        Args:
            num_users: Total unique users.
            num_items: Total unique items.
            embedding_dim: Latent vector size.
        """
        super(FPMC, self).__init__()
        
        # 1. User Embedding (Long-term preference)
        # Represents 'u' in the formula
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        
        # 2. Item Context Embedding (The 'Previous' Item)
        # Represents 'c_l' (Source/Context role)
        self.item_context_embedding = nn.Embedding(num_items, embedding_dim)
        
        # 3. Item Target Embedding (The 'Next' Item)
        # Represents 'v_i' (Destination/Target role)
        # Used for both the MF part and the MC part
        self.item_target_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Initialization
        # FPMC is sensitive to initialization. We use small random values.
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.item_context_embedding.weight, std=0.01)
        nn.init.normal_(self.item_target_embedding.weight, std=0.01)

    def forward(self, user_indices, last_item_indices, curr_item_indices):
        """
        Args:
            user_indices: [Batch_Size]
            last_item_indices: [Batch_Size]
            curr_item_indices: [Batch_Size]
            
        Returns:
            scores: [Batch_Size]
        """
        # 1. Lookup Embeddings
        # Shapes: [Batch, Dim]
        u = self.user_embedding(user_indices)
        c = self.item_context_embedding(last_item_indices)
        v = self.item_target_embedding(curr_item_indices)
        
        # 2. Compute Score
        # Formula: (User + Context) dot Target
        # This efficiently calculates (User dot Target) + (Context dot Target)
        interaction_vector = u + c
        scores = (interaction_vector * v).sum(dim=1)
        
        return scores

In [7]:
import torch
import numpy as np
import random
import math

class FPMCEvaluator:
    def __init__(self, eval_data, exclusion_rules, num_items, k_list=[1, 10]):
        """
        Args:
            eval_data: List/Dict containing 'sequence' (for context) and 'target' (ground truth).
            exclusion_rules: Dict {user_idx: set(all_positive_items)}.
            num_items: Catalog size.
            k_list: Metrics to calculate.
        """
        self.eval_data = eval_data
        self.exclusion_rules = exclusion_rules
        self.num_items = num_items
        self.k_list = k_list
    
    def evaluate(self, model, device='cpu'):
        model.eval()
        
        hr_results = {k: [] for k in self.k_list}
        ndcg_results = {k: [] for k in self.k_list}
        
        with torch.no_grad():
            # Iterate using .items() if it's a dict, or enumerate if list
            # Handling both based on your previous data snippets
            iterator = self.eval_data.items() if isinstance(self.eval_data, dict) else enumerate(self.eval_data)
            
            for u_idx, entry in iterator:
                
                # 1. Get Context (The Last Item user interacted with)
                # This is the "Markov" state
                seq = entry['sequence']
                if len(seq) == 0:
                    continue # Skip empty users
                
                last_item_raw = seq[-1]['item_id']
                last_item = last_item_raw - 1
                
                # 2. Get Ground Truth (Target)
                gt_item_raw = entry['target']['item_id']
                gt_item = gt_item_raw - 1
                
                # 3. Sample 100 Negatives
                negatives = []
                u_exclusion = self.exclusion_rules[u_idx]
                
                while len(negatives) < 100:
                    neg_candidate = random.randint(1, self.num_items)
                    # Exclude history AND target
                    if (neg_candidate not in u_exclusion) and (neg_candidate - 1 != gt_item):
                        negatives.append(neg_candidate - 1)
                
                # 4. Prepare Batch (101 Items)
                candidate_items = [gt_item] + negatives
                
                # Create Tensors
                # User: Repeated 101 times
                user_tensor = torch.tensor([u_idx] * 101, dtype=torch.long).to(device)
                
                # Last Item: Repeated 101 times (Context is constant for this prediction)
                last_item_tensor = torch.tensor([last_item] * 101, dtype=torch.long).to(device)
                
                # Current Items: The 101 candidates
                curr_item_tensor = torch.tensor(candidate_items, dtype=torch.long).to(device)
                
                # 5. Score
                # Model takes (u, last, curr)
                scores = model(user_tensor, last_item_tensor, curr_item_tensor)
                scores = scores.cpu().numpy()
                
                # 6. Rank
                ranked_indices = np.argsort(-scores) # Descending
                gt_rank = np.where(ranked_indices == 0)[0][0]
                
                # 7. Metrics
                for k in self.k_list:
                    if gt_rank < k:
                        hr_results[k].append(1)
                        ndcg_results[k].append(1 / math.log2(gt_rank + 2))
                    else:
                        hr_results[k].append(0)
                        ndcg_results[k].append(0)

        avg_hr = {k: np.mean(v) for k, v in hr_results.items()}
        avg_ndcg = {k: np.mean(v) for k, v in ndcg_results.items()}
        
        return avg_hr, avg_ndcg

In [8]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

# --- Hyperparameters ---
BATCH_SIZE = 128
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
EMBEDDING_DIM = 64
N_NEG = 1  # Start with 1 as discussed
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device selected: {DEVICE}")

# --- 1. Setup Data & Model ---
# Ensure item2id and data dicts are available in your scope
num_items_total = len(item2id)

# Dataset
train_dataset_fpmc = FPMCDataset(
    train_data, val_data, test_data, 
    num_items=num_items_total, 
    n_neg=N_NEG
)

# Evaluator
# Uses the same exclusion rules logic
val_evaluator_fpmc = FPMCEvaluator(
    val_data, 
    train_dataset_fpmc.exclusion_rules, 
    num_items=num_items_total, 
    k_list=[1, 10]
)

# DataLoader
train_loader = DataLoader(train_dataset_fpmc, batch_size=BATCH_SIZE, shuffle=True)

# Model
model = FPMC(num_users=len(train_data), num_items=num_items_total, embedding_dim=EMBEDDING_DIM)
model.to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- 2. Training Loop ---
print("Starting FPMC Training...")
print("=" * 60)

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    total_loss = 0.0
    
    # Progress Bar
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS}", leave=False)
    
    for u, last_item, pos_item, neg_items in progress_bar:
        # Move to Device
        u = u.to(DEVICE)                  # [Batch]
        last_item = last_item.to(DEVICE)  # [Batch]
        pos_item = pos_item.to(DEVICE)    # [Batch]
        neg_items = neg_items.to(DEVICE)  # [Batch, N_Neg]
        
        optimizer.zero_grad()
        
        # --- A. Positive Scores ---
        # Shape: [Batch]
        pos_scores = model(u, last_item, pos_item)
        
        # --- B. Negative Scores (Handling N_Neg) ---
        # We must flatten the batch to process (Batch * N_Neg) items at once
        
        # 1. Flatten Negatives: [Batch, N_Neg] -> [Batch * N_Neg]
        neg_items_flat = neg_items.view(-1)
        
        # 2. Repeat User and Last Item to match the flattened negatives
        # [Batch] -> [Batch * N_Neg]
        u_flat = u.repeat_interleave(N_NEG)
        last_item_flat = last_item.repeat_interleave(N_NEG)
        
        # 3. Compute Scores
        neg_scores_flat = model(u_flat, last_item_flat, neg_items_flat)
        
        # 4. Reshape back to [Batch, N_Neg]
        neg_scores = neg_scores_flat.view(-1, N_NEG)
        
        # --- C. BPR Loss ---
        # pos_scores: [Batch] -> [Batch, 1] for broadcasting
        # neg_scores: [Batch, N_Neg]
        # Loss: -log_sigmoid(pos - neg)
        loss = -torch.mean(torch.nn.functional.logsigmoid(pos_scores.unsqueeze(1) - neg_scores))
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})
        
    avg_loss = total_loss / len(train_loader)
    
    # --- 3. Evaluation ---
    hr, ndcg = val_evaluator_fpmc.evaluate(model, device=DEVICE)
    
    # --- 4. Logging ---
    print(f"Epoch {epoch:02d} Completed")
    print("-" * 30)
    print(f"  Training Loss: {avg_loss:.4f}")
    print(f"  Validation HR@1:   {hr[1]:.4f} | HR@10:   {hr[10]:.4f}")
    print(f"  Validation NDCG@1: {ndcg[1]:.4f} | NDCG@10: {ndcg[10]:.4f}")
    print("=" * 60)

# --- 5. Save Model ---
save_path = "/data/sukhanna/cse258/fpmc_model.pth"
torch.save(model.state_dict(), save_path)
print(f"\nFPMC Training Complete. Model saved to: {save_path}")

Device selected: cuda
Starting FPMC Training...


Epoch 01 Completed
------------------------------
  Training Loss: 0.6503
  Validation HR@1:   0.2125 | HR@10:   0.4518
  Validation NDCG@1: 0.2125 | NDCG@10: 0.3246


Epoch 02 Completed
------------------------------
  Training Loss: 0.4144
  Validation HR@1:   0.2432 | HR@10:   0.5234
  Validation NDCG@1: 0.2432 | NDCG@10: 0.3734


Epoch 03 Completed
------------------------------
  Training Loss: 0.2258
  Validation HR@1:   0.2582 | HR@10:   0.5459
  Validation NDCG@1: 0.2582 | NDCG@10: 0.3918


Epoch 04 Completed
------------------------------
  Training Loss: 0.1197
  Validation HR@1:   0.2660 | HR@10:   0.5562
  Validation NDCG@1: 0.2660 | NDCG@10: 0.4013


...

Epoch 06 Completed
------------------------------
  Training Loss: 0.0403
  Validation HR@1:   0.2771 | HR@10:   0.5673
  Validation NDCG@1: 0.2771 | NDCG@10: 0.4131


Epoch 07 Completed
------------------------------
  Training Loss: 0.0263
  Validation HR@1:   0.2808 | HR@10:   0.5690
  Validation NDCG@1: 0.2808 | NDCG@10: 0.4164


Epoch 08 Completed
------------------------------
  Training Loss: 0.0195
  Validation HR@1:   0.2824 | HR@10:   0.5713
  Validation NDCG@1: 0.2824 | NDCG@10: 0.4183


Epoch 09 Completed
------------------------------
  Training Loss: 0.0148
  Validation HR@1:   0.2865 | HR@10:   0.5705
  Validation NDCG@1: 0.2865 | NDCG@10: 0.4203


Epoch 10 Completed
------------------------------
  Training Loss: 0.0123
  Validation HR@1:   0.2889 | HR@10:   0.5724
  Validation NDCG@1: 0.2889 | NDCG@10: 0.4221

FPMC Training Complete. Model saved to: /data/sukhanna/cse258/fpmc_model.pth


In [9]:
# 1. Instantiate the Test Evaluator
# We reuse the exclusion_rules from the training dataset.
# This ensures that for every user, we exclude (Train History + Val Target + Test Target)
# from the pool of 100 negative samples.
test_evaluator_fpmc = FPMCEvaluator(
    eval_data=test_data, 
    exclusion_rules=train_dataset_fpmc.exclusion_rules, 
    num_items=len(item2id), 
    k_list=[1, 10]
)

# 2. Run Evaluation
# Ensure the model is in eval mode (handled inside the class, but good practice)
print("Running FPMC Test Evaluation...")
test_hr, test_ndcg = test_evaluator_fpmc.evaluate(model, device=DEVICE)

# 3. Report Results
print("-" * 30)
print(f"Test HR@1:   {test_hr[1]:.4f} | HR@10:   {test_hr[10]:.4f}")
print(f"Test NDCG@1: {test_ndcg[1]:.4f} | NDCG@10: {test_ndcg[10]:.4f}")
print("-" * 30)

Running FPMC Test Evaluation...
------------------------------
Test HR@1:   0.2674 | HR@10:   0.5477
Test NDCG@1: 0.2674 | NDCG@10: 0.3986
------------------------------
